In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b

In [2]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [3]:
market = Market()
speculation_db = ADatabase("btc_spec")
classification_db = ADatabase("btc_spec_classification")
class_spec = ADatabase("btc_class_spec")

In [4]:
prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["ticker"] = "BTC"

In [5]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [6]:
classification_db.connect()
classification_simulation = classification_db.retrieve("sim")
classification_db.disconnect()

In [7]:
simulation["week"] = simulation["week"] + 1
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation.rename(columns={"prediction":"classification_prediction"},inplace=True)

In [8]:
simulation = prices.merge(simulation.drop("adjclose",axis=1),on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose","training_years"],axis=1),on=["year","week","ticker"],how="left")

In [9]:
sim = simulation.dropna()

In [10]:
sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]

In [11]:
sim

,date,open,high,low,close,adjclose,volume,year,quarter,week,...,d3,rolling14,xgb_prediction_y,xgb_score_y,cat_prediction_y,cat_score_y,tf_prediction_y,tf_score_y,classification_prediction,delta
1573,2019-01-07,4078.584961,4092.613525,4020.894043,4025.248291,4025.248291,5.228626e+09,2019,1,2,...,394.129840,5181.828652,0.0,0.452830,1.0,0.452830,1.0,0.471698,True,0.219827
1574,2019-01-08,4028.472168,4109.020996,3996.955322,4030.847900,4030.847900,5.306593e+09,2019,1,2,...,394.129840,5181.828652,0.0,0.452830,1.0,0.452830,1.0,0.471698,True,0.218133
1575,2019-01-09,4031.552002,4068.403564,4022.662842,4035.296387,4035.296387,5.115905e+09,2019,1,2,...,394.129840,5181.828652,0.0,0.452830,1.0,0.452830,1.0,0.471698,True,0.216790
1576,2019-01-10,4034.411377,4064.066650,3659.174561,3678.924561,3678.924561,6.874144e+09,2019,1,2,...,394.129840,5181.828652,0.0,0.452830,1.0,0.452830,1.0,0.471698,True,0.334659
1577,2019-01-11,3674.015381,3713.881836,3653.069824,3687.365479,3687.365479,5.538713e+09,2019,1,2,...,394.129840,5181.828652,0.0,0.452830,1.0,0.452830,1.0,0.471698,True,0.331603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,2022-12-27,16919.291016,16959.845703,16642.072266,16717.173828,16717.173828,1.574858e+10,2022,4,52,...,8.305415,18294.326730,0.0,0.566038,0.0,0.660377,0.0,0.433962,False,0.373354
3024,2022-12-28,16716.400391,16768.169922,16497.556641,16552.572266,16552.572266,1.700571e+10,2022,4,52,...,8.305415,18294.326730,0.0,0.566038,0.0,0.660377,0.0,0.433962,False,0.387011
3025,2022-12-29,16552.322266,16651.755859,16508.683594,16642.341797,16642.341797,1.447224e+10,2022,4,52,...,8.305415,18294.326730,0.0,0.566038,0.0,0.660377,0.0,0.433962,False,0.379529
3026,2022-12-30,16641.330078,16643.427734,16408.474609,16602.585938,16602.585938,1.592916e+10,2022,4,52,...,8.305415,18294.326730,0.0,0.566038,0.0,0.660377,0.0,0.433962,False,0.382832


In [13]:
trades = []
reqs = [0.05,0.1,0.15]
signals = [0.05,0.1,0.15]
values = [True,False]
parameters = []
positions = 1
training_year = 4
for value in values:
    for signal in signals:
        for req in reqs:
            parameter = {"value":value,"signal":signal,"req":req,"training_years":training_year}
            parameters.append(parameter)

In [ ]:
class_spec.connect()
for parameter in tqdm(parameters):
    current_sim = sim[sim["training_years"]==parameter["training_years"]].reset_index()
    iteration_trades = b.speculation_classification_backtest(sim,parameter,start_date,end_date)
    class_spec.store("trades",pd.DataFrame(iteration_trades))
class_spec.disconnect()

  0%|                                                                                                                                                             | 0/18 [00:00<?, ?it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

  6%|████████▎                                                                                                                                            | 1/18 [00:33<09:33, 33.72s/it]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

 11%|████████████████▌                                                                                                                                    | 2/18 [01:07<09:00, 33.81s/it]

...

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

 72%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                         | 13/18 [07:58<03:30, 42.16s/it]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi